In [27]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re

In [28]:
page = requests.get('https://maldita.es/malditobulo/')

In [29]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [30]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [31]:
tags = soup.find_all('article', class_='card card-item half')

todo = list(tags)
#print(todo)

#Comienzo a procesar el texto html y me quedo solo con los títulos

elementos = []
for articulo in todo:
    #print(articulo)

    if (articulo.get_text() != '\n'):
        #print(articulo.get_text().split('\n'))
        elementos.append(articulo.get_text().strip().split('\n'))
    
#print(elementos)

lista_titulos = []
for e in elementos:

    lista_titulos.append(e[-1].strip())
    
'''
print(len(elementos))
print(len(lista_titulos))
print(lista_titulos)
'''
lista_titulos


['No, no han apuñalado a una persona durante una cacerolada en el barrio de Moratalaz (Madrid)',
 'Ni Italia ha descubierto la cura del coronavirus, ni la causa de la enfermedad es una bacteria, ni la OMS ha prohibido hacer autopsias',
 '¿Qué sabemos sobre que "el estado de alarma se prolongará hasta el 30 de junio" por lo que todo lo que se decida hasta esa fecha en relación con la crisis del coronavirus "tendríamos que tragarlo hasta octubre"? El Gobierno ha solicitado una prórroga de 15 días, que ya ha sido aprobada, y en julio y agosto se pueden hacer sesiones plenarias extraordinarias',
 'Bulos y desinformaciones sobre las protestas y caceroladas contra el Gobierno durante el confinamiento por el coronavirus',
 '¿Qué sabemos de la imagen de una persona con un traje de protección que supuestamente manipula una antena 5G? No es un traje antirradiación y el 5G no es peligroso para la salud',
 'No, consumir alimentos alcalinos no ayuda a combatir el coronavirus',
 'No, el Gobierno no 

In [6]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [32]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url}
    
    return article

In [33]:
lista_enlaces = []
suficiente = True
n_pag = 1
n_noticias = 1#Las noticias van en bloques de 24. La carga computacional aumentaria mucho si quisiera que fuera mas preciso
                #Pero para este caso de uso y estudio no es necesario.

#Cargo las URLS.
print('Cargando URLs...')

while (len(lista_enlaces) <= n_noticias-1 and suficiente):
    link =  "https://maldita.es/malditobulo/page/" + str(n_pag) + "/"
    arrayurls= get_urls(link, lista_enlaces)
    n_pag = n_pag + 1

    if (n_pag >=10):
        suficiente=False
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/05/21/moratalazas-apunalamiento-punalada-cacerolada-madrid/', 'https://maldita.es/malditaciencia/2020/05/21/cadena-italia-cura-coronavirus-autopsias-prohibidas/', 'https://maldita.es/malditobulo/2020/05/21/estado-alarma-prorroga-coronavirus-gobierno-sanchez-reales-decretos-hasta-octubre/', 'https://maldita.es/malditobulo/2020/05/21/bulos-protestas-caceroladas-gobierno-confinamiento-coronavirus/', 'https://maldita.es/malditobulo/2020/05/21/foto-traje-antirradiacion-tecnologia-5g-riesgo-salud/', 'https://maldita.es/malditaciencia/2020/05/21/no-consumir-alimentos-alcalinos-no-ayuda-a-combatir-el-coronavirus/', 'https://maldita.es/malditobulo/2020/05/21/gobierno-consejo-ministros-anunciado-subida-irpf-supresion-pagas-extras-funcionarios-pensionistas/', 'https://maldita.es/malditobulo/2020/05/20/video-ministro-educacion-frances-jean-michel-blanquer-videollamada-bano/', 'https://maldita.es/malditobulo/2020/05/20/explosion-bote-gel-hid

In [34]:
lista_articulos

[{'Title': 'No, no han apuñalado a una persona durante una cacerolada en el barrio de Moratalaz (Madrid)',
  'Publication_date': '21/05/2020',
  'Content': 'Nos habéis consultado sobre un supuesto apuñalamiento ocurrido en la tarde del 20 de mayo durante una cacerolada contra el Gobierno por su gestión de la crisis del coronavirus en el barrio madrileño de Moratalaz. El apuñalamiento es un bulo.La Jefatura Superior de Policía de Madrid afirma a Maldita.es que se atendió a una persona por golpes y nos remite a Emergencias Madrid para conocer el tipo de heridas. Según la Jefatura Superior de Policía de Madrid, no hay denuncia de momento en relación con el incidente.Desde Emergencias Madrid confirman a Maldita.es que este miércoles Samur-Protección Civil ha atendido a un varón de 34 años herido leve con contusiones en la cabeza a las 21:20 en la Avenida de Moratalaz número 181 y que fue trasladado al hospital Gregorio Marañón. Emergencias Madrid desmiente que esta persona tuviese "ninguna

In [11]:
'''
page = requests.get('https://maldita.es/malditaciencia/2020/05/21/cadena-italia-cura-coronavirus-autopsias-prohibidas/')

soup3 = BeautifulSoup(page.content, 'html.parser')

title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
print(title)

date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
print(date)

content_list = []
content = soup3.find("div", class_="article-body")
content2 = content.find_all("p", class_="")

for e in content2:
    content_list.append(e.get_text().replace("\xa0", " "))

content = '\n'.join(content_list)
print(content)
'''

Ni Italia ha descubierto la cura de la COVID-19, ni la causa de la enfermedad es una bacteria, ni la OMS ha prohibido hacer autopsias como afirma esta cadena de WhatsApp
21/05/2020
En los últimos días nos habéis enviado muchísimo una cadena que circula por WhatsApp que dice que en Italia ya se ha encontrado la cura para el coronavirus, gracias a que los médicos italianos desobedecieron "la ley mundial de la salud OMS" de no realizar autopsias a los fallecidos por coronavirus. Esta sería la manera en la que habrían hallado que la causa de la COVID-19 "no es un virus" sino "una bacteria que produce la muerte" haciendo que "se formen coágulos en la sangre". Esto es un bulo. Te contamos lo que sabemos al respecto. 
Es cierto que, durante los últimos meses, una investigación italiana ha llevado a cabo 38 pruebas post mortem de los pulmones de pacientes infectados por COVID-19 en los hospitales Luigi Sacco, en Milán, y Papa Giovanni XXIII, en Bérgamo. Los resultados de estos análisis se dier